In [1]:
from llama_index.core import Settings
from llama_index.llms.groq import Groq

/Users/jonatasmartins/Documents/estudos/IA/LlamaIndex/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
llm = Groq(model="llama-3.3-70b-versatile",
           api_key=os.environ.get("GROQ_API_KEY"))

In [3]:
def calcular_imposto_renda(rendimento: float) -> str:
    """
    Calcula o imposto de renda com base no rendimento anual.
    
    Args:
        rendimento (float): Rendimento anual do indivíduo.
        
    Returns:
        str: O valor do imposto devido com base no rendimento
    """
    if rendimento <= 2000:
        return "Você está isento de pagar imposto de renda"
    elif 2000 < rendimento <= 5000:
        imposto = (rendimento - 2000) * 0.10
        return f"O imposto devido é de R$ {imposto:.2f}, base em um rendimento de R$ {rendimento:.2f}"
    elif 5000 < rendimento <= 10000:
        imposto = (rendimento - 5000) * 0.15 + 300
        return f"O imposto devido é de R$ {imposto:.2f}, base em um rendimento de R$ {rendimento:.2f}"
    else:
        imposto = (rendimento - 10000) * 0.20 + 1050
        return f"O imposto devido é de R$ {imposto:.2f}, base em um rendimento de R$ {rendimento:.2f}"

### Convertendo Função em Ferramenta

In [4]:
from llama_index.core.tools import FunctionTool

In [5]:
ferramenta_imposto_renda = FunctionTool.from_defaults(
    fn=calcular_imposto_renda,
    name="Calcular Imposto de Renda",
    description=(
        "Calcula o imposto de renda com base no rendimento anual."
        "Argumento: rendimento (float)."
        "Retorna o valor do imposto devido de acordo com faixas de rendimento"
    )
)

In [6]:
from llama_index.core.agent import FunctionCallingAgentWorker

In [7]:
agent_worker_imposto = FunctionCallingAgentWorker.from_tools(
    tools=[ferramenta_imposto_renda],
    verbose=True,
    allow_parallel_tool_calls=True,
    llm=llm
)

In [8]:
from llama_index.core.agent import AgentRunner

In [9]:
agent_imposto = AgentRunner(agent_worker_imposto)

In [10]:
response = agent_imposto.chat("""
    Qual é o imposto de renda devido por uma pessoa com rendimento
    anual de R$ 7.500?
    """)

Added user message to memory: 
    Qual é o imposto de renda devido por uma pessoa com rendimento
    anual de R$ 7.500?
    
=== Calling Function ===
Calling function: Calcular Imposto de Renda with args: {"rendimento": 7500}
=== Function Output ===
O imposto devido é de R$ 675.00, base em um rendimento de R$ 7500.00
=== LLM Response ===
O imposto de renda devido por uma pessoa com rendimento anual de R$ 7.500 é de R$ 675,00.


In [11]:
response = agent_imposto.chat("Quem foi Machado de Assis?")

Added user message to memory: Quem foi Machado de Assis?
=== LLM Response ===
Machado de Assis foi um escritor, poeta, contista e dramaturgo brasileiro, considerado um dos maiores nomes da literatura brasileira. Ele é conhecido por suas obras-primas, como "Dom Casmurro", "Memórias Póstumas de Brás Cubas" e "Quincas Borba", que são caracterizadas por sua profundidade psicológica, ironia e crítica social. Machado de Assis é frequentemente chamado de "o maior nome da literatura brasileira" e é considerado um dos principais representantes do Realismo e do Simbolismo no Brasil. Ele foi também um dos fundadores da Academia Brasileira de Letras e ocupou a cadeira número 23.


In [12]:
import arxiv 

def consulta_artigos(titulo: str) -> str:
    """Consulta os artigos na base de dados ArXiv e retorna resultados formatados."""
    busca = arxiv.Search(
        query=titulo,
        max_results=5,
        sort_by=arxiv.SortCriterion.Relevance
    )
    
    resultados = [
        f"Título: {artigo.title}\n"
        f"Categoria: {artigo.primary_category}\n"
        f"Link: {artigo.entry_id}\n"
        for artigo in busca.results()
    ]
    
    return "\n\n".join(resultados)

In [13]:
consulta_artigos_tool = FunctionTool.from_defaults(fn=consulta_artigos)

In [14]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [ferramenta_imposto_renda, consulta_artigos_tool],
    verbose=True,
    allow_parallel_tool_calls=False,
    llm=llm
)

In [19]:
agent = AgentRunner(agent_worker)
response = agent.chat("Me retorne artigos sobre Teste de software")

Added user message to memory: Me retorne artigos sobre Teste de software
=== Calling Function ===
Calling function: consulta_artigos with args: {"titulo": "Teste de software"}


/var/folders/t6/zvqzkrzs6zz3rr3vchkmyxfc0000gn/T/ipykernel_55150/1589355692.py:15: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for artigo in busca.results()


=== Function Output ===
Título: Myths and Facts about a Career in Software Testing: A Comparison between Students' Beliefs and Professionals' Experience
Categoria: cs.SE
Link: http://arxiv.org/abs/2311.06201v1


Título: Software Testing and Code Refactoring: A Survey with Practitioners
Categoria: cs.SE
Link: http://arxiv.org/abs/2310.01719v1


Título: Efectividad de técnicas de prueba de software aplicadas por sujetos novicios de pregrado
Categoria: cs.SE
Link: http://arxiv.org/abs/1504.08052v1


Título: Requirements Engineering, Software Testing and Education: A Systematic Mapping
Categoria: cs.SE
Link: http://arxiv.org/abs/2304.13693v1


Título: Social Aspects of Software Testing: Comparative Studies in Asia
Categoria: cs.SE
Link: http://arxiv.org/abs/2205.07781v1

=== LLM Response ===
Esses artigos podem ser úteis para você. Se precisar de mais ajuda, basta perguntar!


### Usando o Tavily

In [23]:
import os 
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [24]:
tavily_key = os.environ.get("TAVILY_API_KEY")

In [25]:
from llama_index.tools.tavily_research import TavilyToolSpec

tavily_tool = TavilyToolSpec(
    api_key=tavily_key
)

In [26]:
tavily_tool_list = tavily_tool.to_tool_list()
for tool in tavily_tool_list:
    print(tool.metadata.name)

search


In [27]:
tavily_tool.search("Me retorne artigos científicos sobre Testes de software", max_results=3)

[Document(id_='db01bb7d-3a02-4e1a-bbd8-8faae7b64977', embedding=None, metadata={'url': 'https://www.cin.ufpe.br/~processos/TAES3/Workshops_Qualidade/Workshop_Qualidade-2008-2/Testes_Baseados_em_Modelo_27112008.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Em 1950, Alan Turing escreveu o primeiro artigo científico que abordava questões sobre teste de software. Ele definia um teste operacional para demonstrar o', path=None, url=None, mimetype=None), image_resource=None, audio_resource=None, video_resource=None, text_template='{metadata_str}\n\n{content}'),
 Document(id_='9c459dec-567a-4893-b653-9ef5624ce602', embedding=None, metadata={'url': 'https://www.nucleodoconhecimento.com.br/tag/testes-de-software'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', m

In [28]:
from llama_index.core.tools import FunctionTool

tavily_tool_function = FunctionTool.from_defaults(
    fn=tavily_tool.search,
    name="Tavily Search",
    description="Busca artigos com Tavily sobre um determinado tópico"
)

In [29]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    tools=[tavily_tool_function],
    verbose=True,
    allow_parallel_tool_calls=False,
    llm=llm
)

In [30]:
agent = AgentRunner(agent_worker)

In [31]:
response = agent.chat("Me retorne artigos sobre Testes de software")

Added user message to memory: Me retorne artigos sobre Testes de software
=== Calling Function ===
Calling function: Tavily Search with args: {"query": "Testes de software", "max_results": 10}
=== Function Output ===
[Document(id_='6fdef678-abb5-4cbd-ae0d-49dc79b8ee9b', embedding=None, metadata={'url': 'https://www.ibm.com/br-pt/topics/software-testing'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Testes de software são o processo de avaliar e verificar se um produto ou aplicativo de software faz o que deveria fazer. Os benefícios de bons testes incluem a prevenção de bugs e a melhoria do desempenho. [...] Os testes de software seguem um processo comum. As tarefas ou etapas incluem definir o ambiente de testes, desenvolver casos de testes, escrever scripts, analisar os resultados dos testes e apresentar relatórios de defeitos.

In [32]:
print(response)

Os testes de software são um processo de avaliar e verificar se um produto ou aplicativo de software faz o que deveria fazer. Os benefícios de bons testes incluem a prevenção de bugs e a melhoria do desempenho. Os testes de software seguem um processo comum, que inclui definir o ambiente de testes, desenvolver casos de testes, escrever scripts, analisar os resultados dos testes e apresentar relatórios de defeitos. Existem diferentes tipos de testes, como testes unitários, de integração, de sistema, de aceitação e de regressão. Além disso, os testes contínuos desempenham um papel crucial na aceleração do desenvolvimento de software, aprimorando a qualidade do código e evitando gargalos dispendiosos.


In [33]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

In [34]:
url = "files/LLM.pdf"
artigo = SimpleDirectoryReader(input_files=[url]).load_data()

Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 50 0 (offset 0)
Ignoring wrong pointing object 52 0 (offset 0)
Ignoring wrong pointing object 54 0 (offset 0)
Ignoring wrong pointing object 56 0 (offset 0)
Ignoring wrong pointing object 58 0 (offset 0)
Ignoring wrong pointing object 70 0 (offset 0)
Ignoring wrong pointing object 72 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)
Ignoring wrong pointing object 91 0 (offset 0)
Ignoring wrong pointing object 103 0 (offset 0)
Ignoring wrong pointing object 108 0 (offset 0)
Ignoring wrong pointing object 149 0 (offset 0)
Ignoring wrong pointing object 155 0 (offset 0)
Ignoring wrong pointing object 158 0 (offset 0)
Ignoring wrong pointing object 160 0 (offset 0)
Ignoring wrong pointing object 163 0 (offset 0)
Ignori

In [35]:
url = "files/LLM_2.pdf"
tutorial = SimpleDirectoryReader(input_files=[url]).load_data()

### Gerar os Embeddings

In [36]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [37]:
Settings.embed_model = HuggingFaceEmbedding(
    model_name = "intfloat/multilingual-e5-large"
)

In [38]:
artigo_index = VectorStoreIndex.from_documents(artigo)
tutorial_index = VectorStoreIndex.from_documents(tutorial)

In [39]:
artigo_index.storage_context.persist(persist_dir="artigo")
tutorial_index.storage_context.persist(persist_dir="tutorial")

### Engine de Busca

In [40]:
from llama_index.core import StorageContext, load_index_from_storage

In [41]:
storage_context = StorageContext.from_defaults(
    persist_dir="artigo"
)
artigo_index = load_index_from_storage(storage_context)

storage_context = StorageContext.from_defaults(
    persist_dir="tutorial"
)
tutorial_index = load_index_from_storage(storage_context)


In [42]:
artigo_engine = artigo_index.as_query_engine(similarity_top_k=3, llm=llm)
tutorial_engine = tutorial_index.as_query_engine(similarity_top_k=3, llm=llm)

In [43]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [44]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=artigo_engine,
        metadata=ToolMetadata(
            name="artigo_engine",
            description=(
                "Fornece informações sobre LLM e LangChain."
                "Use uma pergunta detalhada em texto simples como entrada para a ferramenta"
            )
        )
    ),
    QueryEngineTool(
        query_engine=tutorial_engine,
        metadata=ToolMetadata(
            name="tutorial_engine",
            description=(
                "Fornece informações sobre casos de uso e aplicações em LLMs."
                "Use uma pergunta detalhada em texto simples como entrada para a ferramenta"
            )
        )
    ),
]

In [45]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    query_engine_tools,
    verbose=True,
    allow_parallel_tool_calls=True,
    llm=llm
)

agent_document = AgentRunner(agent_worker)

In [46]:
response = agent_document.chat(
    "Quais as principais aplicações posso construir com LLM e LangChain?"
)

Added user message to memory: Quais as principais aplicações posso construir com LLM e LangChain?
=== Calling Function ===
Calling function: tutorial_engine with args: {"input": "Quais as principais aplica\u00e7\u00f5es posso construir com LLM e LangChain?"}
=== Function Output ===
As principais aplicações que você pode construir com LLM (Large Language Models) e LangChain incluem:

1. **Criação e aprimoramento de conteúdo**: Geração automática de texto, assistência na redação, tradução automática, resumo de textos, planejamento e roteiro de conteúdo, brainstorming e programação.
2. **Análise e organização de informações**: Análise de sentimento, extração de informações, classificação de textos, revisão técnica e extração de dados específicos de documentos grandes.
3. **Interação e automação**: Chatbots, perguntas e respostas, geração de respostas a perguntas com base em um corpus, e interação com chatbots com entrada não apenas de texto, mas também de imagem, áudio e vídeo.
4. **Casos

In [47]:
response = agent_document.chat(
    "Quais as principais tendências em LangChain e LLM?"
)

Added user message to memory: Quais as principais tendências em LangChain e LLM?
=== Calling Function ===
Calling function: artigo_engine with args: {"input": "Quais as principais tend\u00e7as em LangChain e LLM?"}
=== Function Output ===
As principais tendências em LangChain e LLM incluem o uso de modelos de código aberto, como os transformadores da Hugging Face, que permitem que os usuários ajustem os modelos aos seus próprios dados e os usem de forma personalizada. Além disso, há uma tendência em tornar os LLMs mais acessíveis e fáceis de usar, com melhorias em frameworks como o MLflow. Outra tendência é a importância de ter uma base sólida de dados para implementar e usar os LLMs de forma eficaz. A capacidade de treinar os modelos com dados específicos e ajustá-los a domínios específicos também é uma tendência importante.
=== Calling Function ===
Calling function: artigo_engine with args: {"input": "Quais as principais tend\u00e7as em LangChain e LLM?"}
=== Function Output ===
As p

### Agente ReAct

In [48]:
from llama_index.core.agent import ReActAgent

In [49]:
agent = ReActAgent.from_tools(
    query_engine_tools,
    verbose=True,
    llm=llm
)

In [50]:
response = agent.chat("Quais as principais ferramentas usadas em LangChain?")

> Running step 6ace7110-c441-46ca-b1ff-99c0e64331ef. Step input: Quais as principais ferramentas usadas em LangChain?
Thought: The current language of the user is: Portuguese. I need to use a tool to help me answer the question.
Action: artigo_engine
Action Input: {'input': 'Quais as principais ferramentas usadas em LangChain?'}
Observation: Não há menção às principais ferramentas usadas em LangChain no contexto fornecido. O contexto discute grandes modelos de linguagem (LLM), serviços proprietários, modelos de código aberto e o desenvolvimento histórico dos LLMs, mas não menciona LangChain ou suas ferramentas.
> Running step a6293fc1-cfac-4d34-ad8a-e5327476a342. Step input: None
Thought: A ferramenta artigo_engine não forneceu informações suficientes sobre as principais ferramentas usadas em LangChain. Vou tentar novamente com a ferramenta tutorial_engine.
Action: tutorial_engine
Action Input: {'input': 'Quais as principais ferramentas usadas em LangChain?'}
Observation: Não há menção

In [51]:
response = agent.chat("Quais as principais tendências em LangChain que eu deveria estudar?")

> Running step c5cbfe89-8860-474a-b057-7b4d47007201. Step input: Quais as principais tendências em LangChain que eu deveria estudar?
Thought: A pergunta é sobre tendências em LangChain, então eu preciso usar uma ferramenta para obter informações sobre o assunto.
Action: artigo_engine
Action Input: {'input': 'principais tendências em LangChain', 'type': 'string'}
Observation: Não há informações sobre LangChain nos textos fornecidos. No entanto, é possível identificar algumas tendências relacionadas aos grandes modelos de linguagem (LLMs) que podem estar relacionadas ao contexto mais amplo:

1. **Aumento da acessibilidade**: Com o lançamento de interfaces web simples, como o ChatGPT, os LLMs estão se tornando mais acessíveis a um público mais amplo.
2. **Melhoria dos dados de treinamento**: A coleta e análise de grandes volumes de dados de alta qualidade estão melhorando drasticamente o desempenho dos modelos.
3. **Avanços em técnicas de treinamento**: A integração do feedback humano no 